### Сложное домашнее задание: Реализация системы регистрации и аутентификации для интернет-магазина

**Задача:** В рамках этого задания вы должны реализовать систему регистрации, входа, и управления пользователями для интернет-магазина с использованием Django. Важные аспекты включают в себя: использование встроенной системы аутентификации Django, создание кастомной модели пользователя, отправку подтверждающих писем после регистрации, и интеграцию с корзиной покупок.

#### Часть 1: Кастомная модель пользователя

1. **Создайте кастомную модель пользователя**, которая будет расширять стандартную модель пользователя Django. Добавьте к ней дополнительные поля, такие как:
   - Номер телефона.
   - Адрес доставки.
   - Подтверждение электронной почты.

2. **Используйте `AbstractBaseUser` и `BaseUserManager`** для полной кастомизации модели, а также настроек регистрации и аутентификации.

#### Часть 2: Регистрация и вход

1. **Создайте форму регистрации**:
   - Поля: имя пользователя, пароль, подтверждение пароля, email, телефон.
   - Убедитесь, что email уникален и добавьте проверку на его формат.
   - При успешной регистрации пользователь должен получать письмо для подтверждения email.

2. **Создайте механизм подтверждения email**:
   - После регистрации пользователь должен получить письмо с ссылкой для активации аккаунта.
   - Ссылка должна содержать уникальный токен для подтверждения.

3. **Реализуйте форму входа**:
   - Поля: email, пароль.
   - Вход возможен только после подтверждения электронной почты.

4. **Добавьте возможность сброса пароля**:
   - Реализуйте функцию отправки письма для сброса пароля.
   - Пользователь должен иметь возможность установить новый пароль, если забыл старый.

#### Часть 3: Личный кабинет пользователя

1. **Создайте личный кабинет пользователя**, где он сможет:
   - Просматривать и редактировать свою личную информацию (телефон, адрес).
   - Просматривать историю заказов.
   - Изменять пароль.

2. **Реализуйте возможность удаления аккаунта**:
   - В личном кабинете добавьте кнопку удаления аккаунта. Подтверждение через email для безопасности.

#### Часть 4: Интеграция с корзиной покупок

1. **Свяжите корзину с пользователем**:
   - Когда пользователь не авторизован, корзина должна быть связана с сессией.
   - После авторизации все товары из сессионной корзины должны быть перенесены в корзину пользователя.

2. **Сохранение незавершённого заказа**:
   - Если пользователь начал добавлять товары в корзину, но не завершил покупку, корзина должна оставаться активной после повторного входа.

---

### Шаг 1: Создание кастомной модели пользователя

Для этого шага нам нужно создать свою модель пользователя и менеджера пользователя. Мы будем использовать `AbstractBaseUser`, чтобы кастомизировать поля модели, такие как email и телефон.

#### Модель пользователя:

```python
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, BaseUserManager
from django.utils.translation import gettext_lazy as _

class CustomUserManager(BaseUserManager):
    def create_user(self, email, password=None, **extra_fields):
        if not email:
            raise ValueError(_('The Email field must be set'))
        email = self.normalize_email(email)
        user = self.model(email=email, **extra_fields)
        user.set_password(password)
        user.save(using=self._db)
        return user

    def create_superuser(self, email, password=None, **extra_fields):
        extra_fields.setdefault('is_staff', True)
        extra_fields.setdefault('is_superuser', True)

        return self.create_user(email, password, **extra_fields)

class CustomUser(AbstractBaseUser):
    email = models.EmailField(unique=True, verbose_name='Email')
    phone_number = models.CharField(max_length=15, blank=True, verbose_name='Номер телефона')
    address = models.CharField(max_length=255, blank=True, verbose_name='Адрес доставки')
    is_active = models.BooleanField(default=False, verbose_name='Аккаунт активен')
    is_staff = models.BooleanField(default=False, verbose_name='Сотрудник')
    
    objects = CustomUserManager()

    USERNAME_FIELD = 'email'
    REQUIRED_FIELDS = []

    def __str__(self):
        return self.email
```

- **Поля модели:**
  - `email`: Основное поле для аутентификации.
  - `phone_number`: Номер телефона для связи.
  - `address`: Адрес доставки для заказов.
  - `is_active`: Указывает, активен ли аккаунт (подтверждён ли email).
  - `is_staff`: Указывает, является ли пользователь сотрудником или администратором.

- **Менеджер пользователя**: `CustomUserManager` создаёт пользователя и суперпользователя.

- **USERNAME_FIELD**: Устанавливаем email в качестве основного поля для аутентификации.

---

### Шаг 2: Регистрация и подтверждение email

#### Форма регистрации:

```python
from django import forms
from django.contrib.auth.forms import UserCreationForm
from .models import CustomUser

class CustomUserCreationForm(UserCreationForm):
    email = forms.EmailField(required=True)
    phone_number = forms.CharField(required=False)
    address = forms.CharField(required=False)

    class Meta:
        model = CustomUser
        fields = ('email', 'phone_number', 'address', 'password1', 'password2')

    def clean_email(self):
        email = self.cleaned_data.get('email')
        if CustomUser.objects.filter(email=email).exists():
            raise forms.ValidationError('Пользователь с таким email уже существует.')
        return email
```

#### Вьюха для регистрации и отправки подтверждающего письма:

```python
from django.shortcuts import render, redirect
from django.core.mail import send_mail
from django.utils.http import urlsafe_base64_encode
from django.utils.encoding import force_bytes
from django.contrib.auth.tokens import default_token_generator
from .models import CustomUser
from .forms import CustomUserCreationForm

def register(request):
    if request.method == 'POST':
        form = CustomUserCreationForm(request.POST)
        if form.is_valid():
            user = form.save(commit=False)
            user.is_active = False  # Аккаунт не активен до подтверждения email
            user.save()

            # Генерация токена для подтверждения email
            token = default_token_generator.make_token(user)
            uid = urlsafe_base64_encode(force_bytes(user.pk))
            confirmation_link = request.build_absolute_uri(f'/activate/{uid}/{token}/')

            # Отправка email с подтверждением
            send_mail(
                'Подтверждение регистрации',
                f'Перейдите по ссылке для активации: {confirmation_link}',
                'admin@shop.com',
                [user.email]
            )

            return redirect('email_confirmation_sent')
    else:
        form = CustomUserCreationForm()

    return render(request, 'register.html', {'form': form})
```

- **Описание:**
  - Пользователь регистрируется через форму, данные сохраняются, но пользователь остаётся неактивным, пока не подтвердит email.
  - Письмо с ссылкой для активации отправляется на указанный адрес электронной почты.

#### Вьюха активации:

```python
from django.shortcuts import get_object_or_404, redirect
from django.utils.http import urlsafe_base64_decode
from django.contrib.auth.tokens import default_token_generator
from .models import CustomUser

def activate(request, uidb64, token):
    try:
        uid = urlsafe_base64_decode(uidb64).decode()
        user = CustomUser.objects.get(pk=uid)
    except (TypeError, ValueError, OverflowError, CustomUser.DoesNotExist):
        user = None

    if user is not None and default_token_generator.check_token(user, token):
        user.is_active = True
        user.save()
        return redirect('login')
    else:
        return render(request, 'activation_invalid.html')
```

- **Описание:**
  - Когда пользователь переходит по ссылке, система проверяет токен и активирует аккаунт.

---

### Шаг 3: Личный кабинет

#### Вьюха личного кабинета:

```python
from django.contrib.auth.decorators import login_required
from django.shortcuts import render

@login_required
def profile(request):
    return render(request, 'profile.html')
```

- **Описание:**
  - Пользователь может получить доступ к личному кабинету только после авторизации. Здесь можно отображать информацию, такие как история заказов, информация о пользователе, и предоставлять возможность изменить пароль.

---

### Шаг 4: Интеграция с корзиной

1. **Корзина для анонимных пользователей сохраняется в сессии**. После регистрации или входа корзина должна быть перенесена в аккаунт пользователя.

#### Вьюха для переноса корзины:

```python
from django.contrib.auth import login
from django.shortcuts import redirect
from .models import Cart

def login_and_merge_cart(request, user):
    # Логика объединения корзины сессии и корзины пользователя
    if 'cart' in request.session:
        cart = Cart.objects.get(user=user)
        # Переносим данные из сессионной корзины в корзину пользователя
        session_cart = request.session['cart']
        for item

 in session_cart:
            cart.items.create(product_id=item['product_id'], quantity=item['quantity'])
        request.session.pop('cart')
    login(request, user)
    return redirect('profile')
```

- **Описание:**
  - Корзина, созданная для анонимного пользователя, при входе переносится в корзину авторизованного пользователя.

---

### Результат:

Это сложное домашнее задание поможет вам реализовать полную систему аутентификации и регистрации для интернет-магазина, с возможностью подтверждения email, восстановлением пароля, личным кабинетом, и интеграцией с корзиной покупок.